In [3]:
!pip -q install -U langgraph langchain langchain-google-genai google-generativeai typing_extensions
!pip -q install "requests==2.32.4"

import os
import json
import textwrap
import getpass
import time
from typing import Annotated, List, Dict, Any, Optional

from typing_extensions import TypedDict

from langchain.chat_models import init_chat_model
from langchain_core.messages import BaseMessage
from langchain_core.tools import tool

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.checkpoint.memory import InMemorySaver
from langgraph.prebuilt import ToolNode, tools_condition

import requests
from requests.adapters import HTTPAdapter, Retry

if not os.environ.get("GOOGLE_API_KEY"):
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("🔑 Enter your Google API Key (Gemini): ")

llm = init_chat_model("google_genai:gemini-2.0-flash")

In [4]:
WIKI_SEARCH_URL = "https://en.wikipedia.org/w/api.php"

_session = requests.Session()
_session.headers.update({
    "User-Agent": "LangGraph-Colab-Demo/1.0 (contact: example@example.com)",
    "Accept": "application/json",
})
retry = Retry(
    total=5, connect=5, read=5, backoff_factor=0.5,
    status_forcelist=(429, 500, 502, 503, 504),
    allowed_methods=("GET", "POST")
)
_session.mount("https://", HTTPAdapter(max_retries=retry))
_session.mount("http://", HTTPAdapter(max_retries=retry))

def _wiki_search_raw(query: str, limit: int = 3) -> List[Dict[str, str]]:
    """
    Use MediaWiki search API with:
      - origin='*' (good practice for CORS)
      - Polite UA + retries
    Returns compact list of {title, snippet_html, url}.
    """
    params = {
        "action": "query",
        "list": "search",
        "format": "json",
        "srsearch": query,
        "srlimit": limit,
        "srprop": "snippet",
        "utf8": 1,
        "origin": "*",
    }
    r = _session.get(WIKI_SEARCH_URL, params=params, timeout=15)
    r.raise_for_status()
    data = r.json()
    out = []
    for item in data.get("query", {}).get("search", []):
        title = item.get("title", "")
        page_url = f"https://en.wikipedia.org/wiki/{title.replace(' ', '_')}"
        snippet = item.get("snippet", "")
        out.append({"title": title, "snippet_html": snippet, "url": page_url})
    return out

@tool
def wiki_search(query: str) -> List[Dict[str, str]]:
    """Search Wikipedia and return up to 3 results with title, snippet_html, and url."""
    try:
        results = _wiki_search_raw(query, limit=3)
        return results if results else [{"title": "No results", "snippet_html": "", "url": ""}]
    except Exception as e:
        return [{"title": "Error", "snippet_html": str(e), "url": ""}]

TOOLS = [wiki_search]

In [5]:
class State(TypedDict):
    messages: Annotated[list, add_messages]

graph_builder = StateGraph(State)

llm_with_tools = llm.bind_tools(TOOLS)

SYSTEM_INSTRUCTIONS = textwrap.dedent("""
You are ResearchBuddy, a careful research assistant.
- If the user asks you to "research", "find info", "latest", "web", or references a library/framework/product,
  you SHOULD call the `wiki_search` tool at least once before finalizing your answer.
- When you call tools, be concise in the text you produce around the call.
- After receiving tool results, cite at least the page titles you used in your summary.
""").strip()

def chatbot(state: State) -> Dict[str, Any]:
    """Single step: call the LLM (with tools bound) on the current messages."""
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

graph_builder.add_node("chatbot", chatbot)

tool_node = ToolNode(tools=TOOLS)
graph_builder.add_node("tools", tool_node)

graph_builder.add_conditional_edges("chatbot", tools_condition)
graph_builder.add_edge("tools", "chatbot")
graph_builder.add_edge(START, "chatbot")

memory = InMemorySaver()
graph = graph_builder.compile(checkpointer=memory)

In [6]:
def print_last_message(event: Dict[str, Any]):
    """Pretty-print the last message in an event if available."""
    if "messages" in event and event["messages"]:
        msg = event["messages"][-1]
        try:
            if isinstance(msg, BaseMessage):
                msg.pretty_print()
            else:
                role = msg.get("role", "unknown")
                content = msg.get("content", "")
                print(f"\n[{role.upper()}]\n{content}\n")
        except Exception:
            print(str(msg))

def show_state_history(cfg: Dict[str, Any]) -> List[Any]:
    """Print a concise view of checkpoints; return the list as well."""
    history = list(graph.get_state_history(cfg))
    print("\n=== 📜 State history (most recent first) ===")
    for i, st in enumerate(history):
        n = st.next
        n_txt = f"{n}" if n else "()"
        print(f"{i:02d}) NumMessages={len(st.values.get('messages', []))}  Next={n_txt}")
    print("=== End history ===\n")
    return history

def pick_checkpoint_by_next(history: List[Any], node_name: str = "tools") -> Optional[Any]:
    """Pick the first checkpoint whose `next` includes a given node (e.g., 'tools')."""
    for st in history:
        nxt = tuple(st.next) if st.next else tuple()
        if node_name in nxt:
            return st
    return None

In [7]:
config = {"configurable": {"thread_id": "demo-thread-1"}}

first_turn = {
    "messages": [
        {"role": "system", "content": SYSTEM_INSTRUCTIONS},
        {"role": "user", "content": "I'm learning LangGraph. Could you do some research on it for me?"},
    ]
}

print("\n==================== 🟢 STEP 1: First user turn ====================")
events = graph.stream(first_turn, config, stream_mode="values")
for ev in events:
    print_last_message(ev)

second_turn = {
    "messages": [
        {"role": "user", "content": "Ya that's helpful. Maybe I'll build an autonomous agent with it!"}
    ]
}

print("\n==================== 🟢 STEP 2: Second user turn ====================")
events = graph.stream(second_turn, config, stream_mode="values")
for ev in events:
    print_last_message(ev)


==================== 🟢 STEP 1: First user turn ====================
================================ Human Message =================================

I'm learning LangGraph. Could you do some research on it for me?
================================== Ai Message ==================================
Tool Calls:
  wiki_search (7c30a0a2-276f-4559-b3cd-718850d129b4)
 Call ID: 7c30a0a2-276f-4559-b3cd-718850d129b4
  Args:
    query: LangGraph
================================= Tool Message =================================
Name: wiki_search

[{"title": "LangChain", "snippet_html": "launched <span class=\"searchmatch\">LangGraph</span> Platform into general availability, providing managed infrastructure for deploying long-running, stateful AI agents. <span class=\"searchmatch\">Lang</span>Chain&#039;s developers", "url": "https://en.wikipedia.org/wiki/LangChain"}, {"title": "C (programming language)", "snippet_html": "(PDF). Archived (PDF) from the original on October 25, 2007. (3.61 MB) comp.<sp

In [8]:
print("\n==================== 🔁 REPLAY: Full state history ====================")
history = show_state_history(config)

to_replay = pick_checkpoint_by_next(history, node_name="tools")
if to_replay is None:
    to_replay = history[min(2, len(history) - 1)]

print("Chosen checkpoint to resume from:")
print("  Next:", to_replay.next)
print("  Config:", to_replay.config)

print("\n==================== ⏪ RESUME from chosen checkpoint ====================")
for ev in graph.stream(None, to_replay.config, stream_mode="values"):
    print_last_message(ev)

MANUAL_INDEX = None
if MANUAL_INDEX is not None and 0 <= MANUAL_INDEX < len(history):
    chosen = history[MANUAL_INDEX]
    print(f"\n==================== 🧭 MANUAL RESUME @ index {MANUAL_INDEX} ====================")
    print("Next:", chosen.next)
    print("Config:", chosen.config)
    for ev in graph.stream(None, chosen.config, stream_mode="values"):
        print_last_message(ev)

print("\n✅ Done. You added steps, replayed history, and resumed from a prior checkpoint.")


==================== 🔁 REPLAY: Full state history ====================

=== 📜 State history (most recent first) ===
00) NumMessages=7  Next=()
01) NumMessages=6  Next=('chatbot',)
02) NumMessages=5  Next=('__start__',)
03) NumMessages=5  Next=()
04) NumMessages=4  Next=('chatbot',)
05) NumMessages=3  Next=('tools',)
06) NumMessages=2  Next=('chatbot',)
07) NumMessages=0  Next=('__start__',)
=== End history ===

Chosen checkpoint to resume from:
  Next: ('tools',)
  Config: {'configurable': {'thread_id': 'demo-thread-1', 'checkpoint_ns': '', 'checkpoint_id': '1f086431-27ee-6bd1-8001-f0def998f65f'}}

==================== ⏪ RESUME from chosen checkpoint ====================
================================== Ai Message ==================================
Tool Calls:
  wiki_search (7c30a0a2-276f-4559-b3cd-718850d129b4)
 Call ID: 7c30a0a2-276f-4559-b3cd-718850d129b4
  Args:
    query: LangGraph
================================= Tool Message =================================
Name: wiki_searc